In [10]:
import pymysql
from dotenv import load_dotenv
import os

load_dotenv()

db = pymysql.connect(
    host='ec2-3-38-45-40.ap-northeast-2.compute.amazonaws.com',
    port=int(os.getenv('DATABASE_PORT')),
    user=os.getenv('DATABASE_USER'),
    passwd=os.getenv('DATABASE_PASSWORD'),
    db=os.getenv('DATABASE_DATABASE'),
    charset='utf8'
    )

con = db.cursor()

In [11]:
sql = "SELECT * FROM FOOD"
con.execute(sql)
results = con.fetchall()
for row in results:
    print(row)

(18, '버섯 두유 소스 볶음', 'http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00018_2.png', '두유와 생크림으로 고소한 맛을 내어 짠맛을 대신해 줘요. 두유는 식사대용으로 먹을 정도로 영양이 우수하며, 두유를 소스에 사용하면 생크림 사용량을 줄일 수 있어 지방섭취를 줄일 수 있어요. 두유 200ml에 125kcal의\n열량이 있으므로 당뇨 환자의 경우 한잔 정도가 적당해요', 8, 12265, '볶기', '버섯볶음\n새송이버섯 100g(3개), 올리브유 10g(2작은술)\n두유 소스\n두유 20g(1⅓큰술), 생크림 10g(2작은술), 청양고추 5g(1/2개), 후추 약간\n곁들임\n새싹채소 20g', 285, 17, 9, 20, 53, None, None)
(19, '백일송이버섯 볶음', 'http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00019_2.png', '파프리카에 함유된 칼륨, 철분, 구리, 아연 성분은 체내 나트륨 수치를 낮추는 효과가 있으며, 백일송이버섯은 나트륨 함량이 높은 생선이나 육류 요리와 함께 섭취하면 나트륨 섭취를 줄일 수 있어요.', 0, 13, '볶기', '백일송이버섯볶음\n백일송이버섯 65g(1컵), 노랑 파프리카 5g(5×1cm), 빨강 파프리카 5g(5×1cm), 카놀라유 3g(2/3작은술)', 80, 10, 3, 3, 11, None, None)
(21, '땅콩소스 버섯가지나물', 'http://www.foodsafetykorea.go.kr/uploadimg/20230309/20230309030010_1678341610669.jpg', '•땅콩과 저염간장으로 만든 소스로 나트륨 섭취를 줄이고 짠맛을 고소한 맛으로 대체했어요.', 0, 0, '끓이기', '•필수 재료 : 가지(15g), 새송이버섯(15g), 표고버섯(10g)\n•땅콩 소스 : 땅콩가루(5g), 다진마늘(2g), 참기름(2g), 저염간장(1g)', 64, 3

In [12]:
import json

with open('./recipe1~1000.json', 'r') as file:
    data = json.load(file)

In [13]:
foodList = data["COOKRCP01"]["row"]
print(foodList[0])

{'RCP_PARTS_DTLS': '새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)\n고명\n시금치 10g(3줄기)', 'RCP_WAY2': '찌기', 'MANUAL_IMG20': '', 'MANUAL20': '', 'RCP_SEQ': '28', 'INFO_NA': '99', 'INFO_WGT': '', 'INFO_PRO': '14', 'MANUAL_IMG13': '', 'MANUAL_IMG14': '', 'MANUAL_IMG15': '', 'MANUAL_IMG16': '', 'MANUAL_IMG10': '', 'MANUAL_IMG11': '', 'MANUAL_IMG12': '', 'MANUAL_IMG17': '', 'MANUAL_IMG18': '', 'MANUAL_IMG19': '', 'INFO_FAT': '17', 'HASH_TAG': '연두부', 'MANUAL_IMG02': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_2.png', 'MANUAL_IMG03': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_3.png', 'RCP_PAT2': '반찬', 'MANUAL_IMG04': '', 'MANUAL_IMG05': '', 'MANUAL_IMG01': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_1.png', 'MANUAL01': '1. 손질된 새우를 끓는 물에 데쳐 건진다.a', 'ATT_FILE_NO_MK': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00028_1.png', 'MANUAL_IMG06': '', 'MANUAL_IMG07': '', 'MANUAL_IMG08': '', 'MANUAL_IMG09': '

In [14]:
foodTypeSet = set()
for i in range(0, len(foodList)):
    foodTypeSet.add(foodList[i]['RCP_WAY2'])
print(foodTypeSet)

{'볶기', '찌기', '기타', '튀기기', '굽기', '끓이기'}


In [6]:
insertFoodSql = "INSERT INTO food(id, name, main_image, description, type, ingredient, like_count, view_count, calorie, carbohydrate, protein, province, salt) VALUES(%s,%s,%s,%s,%s,%s,0,0,%s,%s,%s,%s,%s)"

data = []
for i in range(0, 1000):
    data.append((int(foodList[i]['RCP_SEQ']), 
                 foodList[i]['RCP_NM'],
                 foodList[i]['ATT_FILE_NO_MAIN'],
                 foodList[i]['RCP_NA_TIP'],
                 foodList[i]['RCP_WAY2'],
                 foodList[i]['RCP_PARTS_DTLS'],
                 int(float(foodList[i]['INFO_ENG'])), 
                 int(float(foodList[i]['INFO_CAR'])),
                 int(float(foodList[i]['INFO_PRO'])),
                 int(float(foodList[i]['INFO_FAT'])),
                 int(float(foodList[i]['INFO_NA']))))

try:
    con.executemany(insertFoodSql, data)  # executemany로 데이터 삽입
    db.commit()  # 모든 데이터 삽입 후에 커밋
except Exception as e:
    print(e)
    db.rollback()  # 롤백하여 이전 상태로 복원


In [7]:
insertRecipeSql = "INSERT INTO recipe(id, food, recipeOrder, image, description) VALUES(%s,%s,%s,%s,%s)"
autoIcrId = 1

data = []
for i in range(0, 1000):
    manualIndex = 1
    while(foodList[i]['MANUAL' + str(manualIndex).zfill(2)] != ""):
        data.append((autoIcrId, 
                     int(foodList[i]['RCP_SEQ']),
                     manualIndex,
                     foodList[i]['MANUAL_IMG' + str(manualIndex).zfill(2)],
                     foodList[i]['MANUAL' + str(manualIndex).zfill(2)]))
        autoIcrId += 1
        manualIndex += 1

try:
    con.executemany(insertRecipeSql, data)  # executemany로 데이터 삽입
    db.commit()  # 모든 데이터 삽입 후에 커밋
except Exception as e:
    print(e)
    db.rollback()  # 롤백하여 이전 상태로 복원

In [16]:
updateFoodSql = "UPDATE FOOD SET kind = %s, hashtag = %s WHERE id = %s"

for item in foodList:
    try:
        con.execute(updateFoodSql, (item['RCP_PAT2'], item['HASH_TAG'], int(item['RCP_SEQ'])))
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
